# KMeans

In [6]:
import cluster_data_5d
from cluster_data_5d import run_clustering, normalize_data, unnormalize
import numpy as np
import pandas as pd
import os
from kmeans import k_means
from cluster_plotter import ClusterPlotter
from clustering_utils_5d import ClusterData
import cluster_plotter
import high_dim_analysis
import seaborn as sns
import matplotlib.pyplot as plt

standard_year_ranges = {
    f"{start}-{start + 3}": np.arange(start, start + 4)
    for start in [2002, 2006, 2010, 2014, 2019]
}
standard_year_ranges["2019-2023"] = np.arange(2019, 2024)  # Include 2023
running_ranges = cluster_data_5d.generate_running_year_ranges(2002, 2023, 4)

binned_data = cluster_data_5d.bin_data_for_clustering(running_ranges, print_res=False)

k_values = [5, 6, 7]

results_per_year_range = {}

# Output directory for plots
plot_dir = "Images\\k_means_running_bins_5d_with_perigee"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

for cluster_data, year_range in binned_data:  
    print(f"\nRunning K-Means for Year Range: {year_range}")

    data_array = np.array([cluster_data.inc, cluster_data.raan, cluster_data.ecc]).T
    feature_names = ["inclination [°]", "RAAN [°]", "Eccentricity"]

    normalized_data, data_min, data_max = normalize_data(data_array)

    k_values = [3, 4, 5, 6, 7]

    results = []

    for k in k_values:
        result_kmeans, time_kmeans, n_clusters_kmeans, points_per_cluster_kmeans, metrics_kmeans = run_clustering(
            k_means, f"K-means (k={k})", normalized_data, data_min, data_max, k, init='kmeans++'
        )

        unnormalized_data, cluster_centers = unnormalize(
            result_kmeans.data, result_kmeans.cluster_centers, data_min, data_max
        )

        results.append({
            "Year Range": year_range,
            "k": k,
            "Runtime (s)": f"{time_kmeans:.3f}",
            "Clusters": n_clusters_kmeans,
            "Points per Cluster": points_per_cluster_kmeans,
            "Davies-Bouldin": f"{metrics_kmeans[0]:.3f}" if metrics_kmeans and isinstance(metrics_kmeans[0], (int, float)) else None,
            "Calinski-Harabasz": f"{metrics_kmeans[1]:.3f}" if metrics_kmeans and isinstance(metrics_kmeans[1], (int, float)) else None,
            "Dunn Index": f"{metrics_kmeans[2]:.3f}" if metrics_kmeans and isinstance(metrics_kmeans[2], (int, float)) else None,
            "Silhouette Score": f"{metrics_kmeans[3]:.3f}" if metrics_kmeans and isinstance(metrics_kmeans[3], (int, float)) else None
        })

        labels = result_kmeans.labels
        """high_dim_analysis.plot_co_membership(
            labels,
            output_folder=plot_dir,
            filename=f"co_membership_{year_range}_k{k}.png"
        )"""
        high_dim_analysis.plot_correlation_heatmap(
            cluster_data,
            output_folder=plot_dir,
            filename=f"corr_{year_range}_k{k}.png"
        )
        high_dim_analysis.plot_mutual_information(
            cluster_data,
            labels,
            output_folder=plot_dir,
            filename=f"mi_{year_range}_k{k}.png"
        )

        from sklearn.feature_selection import f_classif
        X = np.array([cluster_data.inc, cluster_data.raan, cluster_data.perigee,
              cluster_data.ecc, cluster_data.mag]).T
        valid = labels != -1  # Exclude noise points (usually not present for kmeans)
        if np.any(valid) and len(np.unique(labels[valid])) > 1:
            f_vals, _ = f_classif(X[valid], labels[valid])
            mean_f = np.mean(f_vals)
        else:
            mean_f = 0.0  # Avoid invalid or single-cluster cases

        results[-1]["Mean ANOVA F"] = f"{mean_f:.3f}"

    df = pd.DataFrame(results)
    df["Mean ANOVA F"] = pd.to_numeric(df["Mean ANOVA F"], errors="coerce")
    df_sorted = df.sort_values(by="Mean ANOVA F", ascending=False)
    best_params = df_sorted.iloc[0]
    best_k = best_params["k"]

    # Re-run clustering with best parameters to get labels
    best_result, _, _, _, _ = run_clustering(
        k_means, f"K-means (k={best_k})", normalized_data, data_min, data_max, int(best_k), init='kmeans++'
    )
    best_labels = best_result.labels

    # Plot pairplot colored by cluster labels and save to plot_dir
    df_plot = pd.DataFrame(data_array, columns=feature_names)
    df_plot['cluster'] = best_labels.astype(str)  # cluster labels as strings for hue

    pairplot = sns.pairplot(df_plot, hue='cluster', diag_kind='kde', plot_kws={'alpha': 0.6, 's': 40})
    pairplot.fig.suptitle(f"K-Means Clusters for Year Range {year_range}", y=1.02)
    pairplot.savefig(os.path.join(plot_dir, f"pairplot_{year_range}_k{best_k}.png"))
    plt.close(pairplot.fig)

    # Plot ANOVA F-values only for best result
    high_dim_analysis.plot_anova_f_values(
        cluster_data,
        best_labels,
        output_folder=plot_dir,
        filename=f"anova_best_{year_range}_k{best_k}.png",
        title=f"ANOVA F-values (Best KMeans Parameters), k = {int(best_k)}"
    )
    print("Best KMeans parameters based on ANOVA F-value:")
    print(best_params)

    display(df)



Running K-Means for Year Range: 2002-2005
Runtime for k_means: 0.016027 seconds


c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:138: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=mi_vals, y=features, palette="magma")


Runtime for k_means: 0.035876 seconds


c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:138: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=mi_vals, y=features, palette="magma")


Runtime for k_means: 0.043093 seconds


...

Runtime for k_means: 0.033998 seconds


c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:138: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=mi_vals, y=features, palette="magma")


Runtime for k_means: 0.007500 seconds
Best KMeans parameters based on ANOVA F-value:
Year Range                                            2020-2023
k                                                             5
Runtime (s)                                               0.000
Clusters                                                      5
Points per Cluster    {0: 543, 1: 743, 2: 605, 3: 2034, 4: 447}
Davies-Bouldin                                             None
Calinski-Harabasz                                          None
Dunn Index                                                 None
Silhouette Score                                           None
Mean ANOVA F                                          10275.825
Name: 2, dtype: object


c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:42: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=f_vals, y=features, palette="crest")


,Year Range,k,Runtime (s),Clusters,Points per Cluster,Davies-Bouldin,Calinski-Harabasz,Dunn Index,Silhouette Score,Mean ANOVA F
0,2020-2023,3,0.000,3,"{0: 2788, 1: 513, 2: 1071}",None,None,None,None,5418.333
1,2020-2023,4,0.007,4,"{0: 759, 1: 2041, 2: 1062, 3: 510}",None,None,None,None,4406.506
2,2020-2023,5,0.000,5,"{0: 543, 1: 743, 2: 605, 3: 2034, 4: 447}",None,None,None,None,10275.825
3,2020-2023,6,0.017,6,"{0: 1123, 1: 543, 2: 605, 3: 447, 4: 923, 5: 731}",None,None,None,None,8516.241
4,2020-2023,7,0.034,7,"{0: 520, 1: 923, 2: 731, 3: 428, 4: 582, 5: 66...",None,None,None,None,7088.059


# DBSCAN

In [5]:
import cluster_data_5d
from cluster_data_5d import run_clustering, normalize_data, unnormalize
import numpy as np
import pandas as pd
import os
from kmeans import k_means
from cluster_plotter import ClusterPlotter
from clustering_utils_5d import ClusterData
import cluster_plotter
from DBSCAN import dbscan_clustering
import high_dim_analysis
import seaborn as sns
import matplotlib.pyplot as plt

# Define year ranges
standard_year_ranges = {
    f"{start}-{start + 3}": np.arange(start, start + 4)
    for start in [2002, 2006, 2010, 2014, 2019]
}
standard_year_ranges["2019-2023"] = np.arange(2019, 2024)  # Include 2023
running_ranges = cluster_data_5d.generate_running_year_ranges(2002, 2023, 4)

binned_data = cluster_data_5d.bin_data_for_clustering(running_ranges, print_res=False)

results_per_year_range = {}

# Output directory for plots
plot_dir = "Images\dbscan_tests_running_bins_5d_with_perigee"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

for cluster_data, year_range in binned_data:  
    print(f"\nRunning DBSCAN for Year Range: {year_range}")

    data_array = np.array([cluster_data.inc, cluster_data.raan, cluster_data.perigee]).T
    feature_names = ["inclination [°]", "RAAN [°]", "Perigee[km]"]

    normalized_data, data_min, data_max = normalize_data(data_array)

    # DBSCAN parameter ranges
    eps_values = [0.02, 0.01, 0.015]
    min_samples_values = [10, 15, 25, 30]

    results = []

    for eps in eps_values:
        for min_samples in min_samples_values:
            result_dbscan, time_dbscan, n_clusters_dbscan, points_per_cluster_dbscan, metrics_dbscan = run_clustering(
                dbscan_clustering, "DBSCAN", normalized_data, data_min, data_max, eps=eps, min_samples=min_samples
            )

            unnormalized_data, _ = unnormalize(
                result_dbscan.data, None, data_min, data_max
            )
            # Count noise points
            noise_points = np.sum(result_dbscan.labels == -1)

            results.append({
                "Year Range": year_range,
                "eps": eps,
                "min_samples": min_samples,
                "Runtime (s)": f"{time_dbscan:.3f}",
                "Clusters": n_clusters_dbscan,
                "Points per Cluster": points_per_cluster_dbscan,
                "Noise Points": noise_points,
                "Davies-Bouldin": f"{metrics_dbscan[0]:.3f}" if metrics_dbscan and isinstance(metrics_dbscan[0], (int, float)) else None,
                "Calinski-Harabasz": f"{metrics_dbscan[1]:.3f}" if metrics_dbscan and isinstance(metrics_dbscan[1], (int, float)) else None
            })


            labels = result_dbscan.labels
            """high_dim_analysis.plot_co_membership(
                labels,
                output_folder=plot_dir,
                filename=f"co_membership_{year_range}_eps{eps}_minsamples_{min_samples}.png"
            )"""
            high_dim_analysis.plot_correlation_heatmap(
                cluster_data,
                output_folder=plot_dir,
                filename=f"corr_{year_range}_eps{eps}_minsamples_{min_samples}.png"
            )
            high_dim_analysis.plot_mutual_information(
                cluster_data,
                labels,
                output_folder=plot_dir,
                filename=f"mi_{year_range}_eps{eps}_minsamples_{min_samples}.png"
            )
            
        from sklearn.feature_selection import f_classif
        X = np.array([cluster_data.inc, cluster_data.raan, cluster_data.perigee,
              cluster_data.ecc, cluster_data.mag]).T
        valid = labels != -1  # Exclude noise points
        if np.any(valid) and len(np.unique(labels[valid])) > 1:
            f_vals, _ = f_classif(X[valid], labels[valid])
            weights = np.var(X[valid], axis=0)
            weighted_mean_f = np.average(f_vals, weights=weights)
        else:
            weighted_mean_f = 0.0  # Avoid invalid or single-cluster cases

        results[-1]["Weighted Mean ANOVA F"] = f"{weighted_mean_f:.3f}"

    df = pd.DataFrame(results)

    df["Weighted Mean ANOVA F"] = pd.to_numeric(df["Weighted Mean ANOVA F"], errors="coerce")
    df_sorted = df.sort_values(by="Weighted Mean ANOVA F", ascending=False)
    best_params = df_sorted.iloc[0]
    best_eps = best_params["eps"]
    best_min_samples = best_params["min_samples"]

    # Re-run clustering with best parameters to get labels
    best_result, _, _, _, _ = run_clustering(
        dbscan_clustering, "DBSCAN", normalized_data, data_min, data_max,
        eps=best_eps, min_samples=best_min_samples
    )
    best_labels = best_result.labels

    df_plot = pd.DataFrame(data_array, columns=feature_names)
    df_plot['cluster'] = best_labels.astype(str)

    pairplot = sns.pairplot(df_plot, hue='cluster', diag_kind='kde', plot_kws={'alpha':0.6, 's':40})
    pairplot.fig.suptitle(f"DBSCAN Clusters for Year Range {year_range}", y=1.02)

    # Save the figure to the plot directory
    pairplot.savefig(os.path.join(plot_dir, f"pairplot_{year_range}_eps{best_eps}_minsamples_{best_min_samples}.png"))

    plt.close(pairplot.fig)  # Close the figure to free memory


    # Plot ANOVA F-values only for best result
    high_dim_analysis.plot_anova_f_values(
        cluster_data,
        best_labels,
        output_folder=plot_dir,
        filename=f"anova_best_{year_range}_eps{best_eps}_minsamples_{best_min_samples}.png",
        title=f"ANOVA F-values (Best DBSCAN Parameters), eps = {float(best_eps)}, minsamples = {float(best_min_samples)}"
    )

    print("Best DBSCAN parameters based on ANOVA F-value:")
    print(best_params)
    display(df)


<>:28: SyntaxWarning: invalid escape sequence '\d'
<>:28: SyntaxWarning: invalid escape sequence '\d'
C:\Users\fionu\AppData\Local\Temp\ipykernel_25276\3495779337.py:28: SyntaxWarning: invalid escape sequence '\d'
  plot_dir = "Images\dbscan_tests_running_bins_5d_with_perigee"



Running DBSCAN for Year Range: 2002-2005
Runtime for dbscan_clustering: 0.050041 seconds


c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:138: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=mi_vals, y=features, palette="magma")


Runtime for dbscan_clustering: 0.111978 seconds


c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:138: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=mi_vals, y=features, palette="magma")


...

c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:138: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=mi_vals, y=features, palette="magma")


Runtime for dbscan_clustering: 0.032418 seconds


c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\AIUB-Backup\clustering\high_dim_analysis.py:42: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=f_vals, y=features, palette="crest")


Best DBSCAN parameters based on ANOVA F-value:
Year Range                               2020-2023
eps                                          0.015
min_samples                                     30
Runtime (s)                                  0.016
Clusters                                         3
Points per Cluster       {0: 467, 1: 33, -1: 3872}
Noise Points                                  3872
Davies-Bouldin                                None
Calinski-Harabasz                             None
Weighted Mean ANOVA F                     82764.02
Name: 11, dtype: object


,Year Range,eps,min_samples,Runtime (s),Clusters,Points per Cluster,Noise Points,Davies-Bouldin,Calinski-Harabasz,Weighted Mean ANOVA F
0,2020-2023,0.020,10,0.046,67,"{0: 10, 1: 10, 2: 494, 3: 25, 4: 29, 5: 25, 6:...",2332,None,None,NaN
1,2020-2023,0.020,15,0.030,35,"{0: 494, 1: 25, 2: 20, 3: 18, 4: 15, 5: 43, 6:...",2826,None,None,NaN
2,2020-2023,0.020,25,0.035,8,"{0: 494, 1: 32, 2: 61, 3: 42, 4: 25, 5: 135, 6...",3546,None,None,NaN
3,2020-2023,0.020,30,0.018,7,"{0: 492, 1: 51, 2: 38, 3: 92, 4: 26, 5: 30, -1...",3643,None,None,35544.301
4,2020-2023,0.010,10,0.011,29,"{0: 10, 1: 451, 2: 14, 3: 11, 4: 10, 5: 13, 6:...",3561,None,None,NaN
5,2020-2023,0.010,15,0.016,8,"{0: 433, 1: 18, 2: 22, 3: 21, 4: 20, 5: 15, 6:...",3828,None,None,NaN
6,2020-2023,0.010,25,0.019,2,"{0: 424, -1: 3948}",3948,None,None,NaN
7,2020-2023,0.010,30,0.016,3,"{0: 381, 1: 32, -1: 3959}",3959,None,None,17.979
8,2020-2023,0.015,10,0.032,52,"{0: 10, 1: 487, 2: 25, 3: 14, 4: 10, 5: 15, 6:...",2942,None,None,NaN
9,2020-2023,0.015,15,0.032,21,"{0: 482, 1: 15, 2: 21, 3: 19, 4: 43, 5: 16, 6:...",3456,None,None,NaN
